This notebook is originally created by Justin Kinney from https://github.com/jbkinney/19_urp and has been modified by Ben Harris for the 2021 version of the course

# Notebook 4: Pandas and TF binding site analysis

Here we will start using Pandas. Pandas is the standard way of working with columnar data. However, there is a substantial learning curve. If you want to learn more about Pandas, here is a useful site: http://pandas.pydata.org/

Here we will use Pandas to analyze transcription factor (TF) binding sites from *Escherichia coli*. We will first focus on CRP, a major regulator in *E. coli* with over 350 functional binding sites.  

## Introduction

Pandas is a library for working with tabular data. I was orignally based on the R data.frame library, but with a slightly different grammer and some different functionality. 

It also can feel really similar to working in SQL.

There are two main types of objects in Pandas.

### Let's orient ourselves with the objects:

1) `pd.Series`
 * A single column of data
 * Contains rownames but no column name, the rownames are always reffered to as `pd.Series.index`
 * Can have an attribute `pd.Series.name` that can serve as the column name
 * Works a lot like a python dictionary

<img src='https://pandas.pydata.org/docs/_images/01_table_series.svg'>

2) `Dataframe`

* A 2-D object
* Each individual column is a Series
* Now it has row (index) and column names
* The orientation of rows vs columns matters a lot
* Generally, you want features as columns and observations as rows


<img src='https://pandas.pydata.org/docs/_images/01_table_dataframe.svg'>


### What are features and Observations?


In [10]:
!pip -q install palmerpenguins
from palmerpenguins import load_penguins
load_penguins().sample(frac=.2)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,year
10,Adelie,Torgersen,37.8,17.1,186.0,3300.0,NaN,2007
258,Gentoo,Biscoe,41.7,14.7,210.0,4700.0,female,2009
242,Gentoo,Biscoe,47.5,15.0,218.0,4950.0,female,2009
51,Adelie,Biscoe,40.1,18.9,188.0,4300.0,male,2008
237,Gentoo,Biscoe,50.8,17.3,228.0,5600.0,male,2009
...,...,...,...,...,...,...,...,...
331,Chinstrap,Dream,52.2,18.8,197.0,3450.0,male,2009
118,Adelie,Torgersen,35.7,17.0,189.0,3350.0,female,2009
112,Adelie,Biscoe,39.7,17.7,193.0,3200.0,female,2009
82,Adelie,Torgersen,36.7,18.8,187.0,3800.0,female,2008


Features are variables, they are the things you measure, whether that be quantitatively or qualitatively

While observations are each data point, in this case it is each penguin
### Things we need to learn 
1) How to create a pandas object/ read data in

2) How to subset your data

3) How to manipulate/mutate your data to create more data

4) How to Summarize or aggregate your data



## Preparing Code 1

In [15]:
## This is needed everytime you run this if you run on google colab
!pip -q install logomaker

In [12]:
# Put this first
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# Import logomaker; we will use this later for visualizing TF motifs.
import logomaker 

In [13]:
# We will be analyzing a standing database of TF binding sites, which is available on RegulonDB. 
# Here is a command for downloading this file (this didn't work in lecture 1)
!wget -O data/binding_site_db.txt http://regulondb.ccg.unam.mx/menu/download/datasets/files/BindingSiteSet.txt 

--2021-06-16 23:11:50--  http://regulondb.ccg.unam.mx/menu/download/datasets/files/BindingSiteSet.txt
Resolving regulondb.ccg.unam.mx (regulondb.ccg.unam.mx)... 132.248.34.160
Connecting to regulondb.ccg.unam.mx (regulondb.ccg.unam.mx)|132.248.34.160|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1288586 (1.2M) [text/plain]
Saving to: ‘data/binding_site_db.txt’

data/binding_site_d 100%[===================>]   1.23M   442KB/s    in 2.8s    

2021-06-16 23:11:53 (442 KB/s) - ‘data/binding_site_db.txt’ saved [1288586/1288586]



In [14]:
# Let's see what this database looks like
!head -n 50 data/binding_site_db.txt

# Copies and Copyright-Notice
#
#      RegulonDB is free for academic/noncommercial use
#
#      User is not entitled to change or erase data sets of the RegulonDB
#      database or to eliminate copyright notices from RegulonDB. Furthermore,
#      User is not entitled to expand RegulonDB or to integrate RegulonDB partly
#      or as a whole into other databank systems, without prior written consent
#      from CCG-UNAM.
#
#      Please check the license at http://regulondb.ccg.unam.mx/menu/download/full_version/terms_and_conditions.jsp
#
# Citation
#
#      Santos-Zavaleta A et al. (2019). RegulonDB v 10.5: tackling challenges to unify classic and high throughput knowledge of gene regulation in E. coli K-12.,
#      Nucleic Acids Res. 2019 Jan 8;47(D1):D212-D220. doi: 10.1093/nar/gky1077.
#
#
# Contact
#
#      Person: RegulonDB Team
#      Web Page: http://regulondb.ccg.unam.mx/menu/about_regulondb/contact_us/index.jsp
#          (regulondb@ccg.unam.mx)
#
#
# Release: 10.8 Date: 10/

## Reading data

When you store your data as a file you can use one of the `pd.read_*` functions to read in data from a variety of different file types

Then to save your progress you can write using `pd.DataFrame.write_*`

<img src='https://pandas.pydata.org/docs/_images/02_io_readwrite.svg'>


There are quite a lot of file types, and they all have their pros/cons. The simpliest ones are `.csv` and `.txt` these are known as flat files and are what we are using today

In [16]:
# To parse this file, use Pandas's method read_csv. 
# We pass the name of this file, as well as other keyword arguments:
#     sep='\t': columns are delimited by tabs
#     comment='#': ignore rows that begin with this
#     header=None: the first row is NOT the name of the columns
# The results are stored as an object known as a dataframe
df = pd.read_csv("data/binding_site_db.txt", sep='\t', comment='#', header=None)
type(df)

pandas.core.frame.DataFrame

In [17]:
# To check that the data has been properly loaded, call the method df.head()
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,ECK125286586,AccB,AccB,ECK125258258,NaN,NaN,forward,ECK125257898,ECK120009421,accBC,-,accBp,NaN,NaN,NaN,"[GEA|W|Gene expression analysis],[IMP|W|Inferr...",Strong
1,ECK120015994,AcrR,AcrR,ECK125202663,1619048.0,1619058.0,forward,ECK125202664,ECK120009381,marRAB,-,marRp,-40.5,catcggtcaaTTCATTCATTTgacttatact,-67.5,"[GEA|W|Gene expression analysis],[CV(GEA)|C|cr...",Confirmed
2,ECK120015994,AcrR,AcrR,ECK125242724,1978422.0,1978432.0,reverse,ECK125242725,ECK120009328,flhDC,-,flhDp,-31.5,agcccccctcCGTTGTATGTGcgtgtagtga,-229.5,"[GEA|W|Gene expression analysis],[APIORCISFBSC...",Strong
3,ECK120015994,AcrR,AcrR,ECK120035040,2313112.0,2313135.0,forward,ECK120035041,ECK120008981,micF,-,micFp,41.0,atttattaccGTCATTCATTTCTGAATGTCTGTTtacccctatt,41.0,[AIBSCS|W|Automated inference based on similar...,Weak
4,ECK120015994,AcrR,AcrR,ECK125202665,4277444.0,4277454.0,forward,ECK125202666,ECK120009640,soxR,-,soxRp,2.5,ataattcctcAAGTTAACTTGaggtaaagcg,-20.5,"[GEA|W|Gene expression analysis],[CV(GEA)|C|cr...",Confirmed


In [18]:
# You get the number of rows and columns from the attribute df.shape
df.shape

(3703, 17)

In [19]:
# We only want the TF name (column 1) and the TF binding site sequence (column 11)
# To keep only these columns, index the df using a list of column names you want (in the order you want)
col_names = [1,11]
df = df[col_names]
df.head()

,1,11
0,AccB,accBp
1,AcrR,marRp
2,AcrR,flhDp
3,AcrR,micFp
4,AcrR,soxRp


In [20]:
# Data frames allow users to give columns meaningful names.
# To rename the columns, set df.columns to a list of the desired names.
df.columns = ['tf','site']
df.head()

,tf,site
0,AccB,accBp
1,AcrR,marRp
2,AcrR,flhDp
3,AcrR,micFp
4,AcrR,soxRp


In [21]:
# We see that some TF sites are listed as NaN. 
# Let's use the dropna() method to get rid of these rows.
df = df.dropna()
df.head()

,tf,site
0,AccB,accBp
1,AcrR,marRp
2,AcrR,flhDp
3,AcrR,micFp
4,AcrR,soxRp


In [22]:
# Note that the last three modifications of df can be accomplished in one line 
df = pd.read_csv("data/binding_site_db.txt", sep='\t', comment='#',
                 header=None, usecols=[1,11], names=['tf','site']).dropna()
df.head()

,tf,site
0,AccB,accBp
1,AcrR,marRp
2,AcrR,flhDp
3,AcrR,micFp
4,AcrR,soxRp


In [23]:
# Check out the pd.read_csv() documentation for a full list
pd.read_csv?

Signature:
pd.read_csv(
    filepath_or_buffer: Union[ForwardRef('PathLike[str]'), str, IO[~T], io.RawIOBase, io.BufferedIOBase, io.TextIOBase, _io.TextIOWrapper, mmap.mmap],
    sep=<object object at 0x10618b9d0>,
    delimiter=None,
    header='infer',
    names=None,
    index_col=None,
    usecols=None,
    squeeze=False,
    prefix=None,
    mangle_dupe_cols=True,
    dtype=None,
    engine=None,
    converters=None,
    true_values=None,
    false_values=None,
    skipinitialspace=False,
    skiprows=None,
    skipfooter=0,
    nrows=None,
    na_values=None,
    keep_default_na=True,
    na_filter=True,
    verbose=False,
    skip_blank_lines=True,
    parse_dates=False,
    infer_datetime_format=False,
    keep_date_col=False,
    date_parser=None,
    dayfirst=False,
    cache_dates=True,
    iterator=False,
    chunksize=None,
    compression='infer',
    thousands=None,
    decimal: str = '.',
    lineterminator=None,
    quotechar='"',
    quoting=0,
    doublequote=True,
 

In [24]:
# Dataframe columns are called 'Series' objects. 
# Essentially, they're numpy arrays with some extra sugar.
col = df['tf']
col.head()

0    AccB
1    AcrR
2    AcrR
3    AcrR
4    AcrR
Name: tf, dtype: object

In [25]:
# You can extract an element from a dataframe by using .loc[]
df.loc[3,'site']

'micFp'

Our goal is to generate sequence logos that represents the binding preferences of TFs in this database.  As a concrete example we'll use CRP, which has a well-characterized binding motif shown here:

<img src="./data/crp_information_logo.png" alt="Drawing" style="width: 700px;"/>

## Subsetting dataframe

In the Zen of Python they say **There should be one-- and preferably only one --obvious way to do it.**

Unfortunately when it comes to subsetting rows and columns in pandas that is not the case

<img src='https://pandas.pydata.org/docs/_images/03_subset_columns.svg' width='700px'>

<img src='https://pandas.pydata.org/docs/_images/03_subset_rows.svg' width='700px'>


In [26]:
# Choose a TF
tf = 'CRP'

# Flag which rows in the dataframe have the correct TF name
flags = (df['tf']==tf)
flags

0       False
1       False
2       False
3       False
4       False
        ...  
3698    False
3699    False
3700    False
3701    False
3702    False
Name: tf, Length: 3703, dtype: bool

In [27]:

# Grab those rows. To be safe use copy() to make sure that, if we
# alter tf_df, df itself doesn't change
tf_df = df[flags].copy()
tf_df.head()

,tf,site
324,CRP,rmfp
325,CRP,ompAp1
326,CRP,agpp
327,CRP,putPp4
328,CRP,putPp4


Selecting rows by a boolean vector, like `flags` is basically the only way I ever subset my rows. When you store rows as observations this is generally the case because you usually are **filtering** the rows based on a specific feature.

### All the ways to subset DataFrames

#### Rows
By boolean : `df[BOOLEAN_VECTOR]` 

By name : `df.loc[LIST_OF_ROW_NAMES, :]`

By Location : `df.iloc[LIST_OF_INTEGERS, :]`



With a function `df.query('COLUMMN==value')` (This one is quite advanced, see [this tutorial for more](https://www.sharpsightlabs.com/blog/pandas-query/))
#### Columns
By boolean : `df.loc[:,BOOLEAN_VECTOR]`

By name** : `df[LIST_OF_COLUMN_NAMES]`

By Location : `df.iloc[:,LIST_OF_INTEGERS]`

With a function : `df.filter(LOTS_OF_OPTIONS)` (This one is quite advanced, see [this tutorial for more](https://www.sharpsightlabs.com/blog/pandas-filter/))

In [ ]:
# Each DNA binding site should be capitalized.
# To do this, we reset 'site' column of tf_df

# Get list of capitalized sites and replace the 'site' column with this
capitalized_sites = [site.upper() for site in tf_df['site']]
tf_df['site'] = capitalized_sites
tf_df.head()

In [ ]:
# In order to derive a motif, all sites we analyze need to be the same length.
# It's good to check that this is actually the case.
# We therefore add a column to tf_df listing the length of each site

# Compute the length of each site and record this in a 'length' column
site_lengths = [len(site) for site in tf_df['site']]
tf_df['length'] = site_lengths
tf_df.head()

## Exercises, part 1

**E4.1.** Use the `unique()` method to determine the lengths of the listed TF binding sites.

In [ ]:
# Answer here

**E4.2.** Apparently these sites have a bunch of different lengths. For a better understanding of this, write a `for` loop to determine how many sites there are of each length.

In [ ]:
# Answer here

## Part 2

In [ ]:
# The 'groupby' method provides a much more convenient way to
# tally up different binding site lengths -- as well as to do a 
# whole host of other computations on data frames. 
tf_df.groupby('length').count()

In [ ]:
# Use the mode() method to compute the most common binding site length
# Note that mode() returns a tuple, of which we need to manually extract the first element
length_mode = tf_df['length'].mode()[0]
length_mode

In [ ]:
# Flag rows having sites of the chosen length
flags = (tf_df['length']==length_mode)

# Only keep these rows
tf_df = tf_df[flags]
tf_df.head()

In [ ]:
# Now extract the 'site' column from tf_df
sites = tf_df['site']
sites.head()

In [ ]:
# Using logomaker.alignment_to_matrix function, compute the number of times each base occurs at each position
# Note that this returns a dataframe
counts_mat = logomaker.alignment_to_matrix(sites)
counts_mat.head()

In [ ]:
# This counts matrix can be visualized as a sequence logo
logomaker.Logo(counts_mat)

## Exercises, part 2

**E4.3.** Counts logos shown above aren't what people use in publications. Rather, they typically use "information" logos, like the one shown earlier. By making use of the keyword argument  `to_type='information'` in the function `logomaker.alignment_to_matrix()`, create a CRP information logo.

In [ ]:
# Answer here

**E4.4.** Using the `.groupby()` and `.sort_values()` methods of the dataframe `df`, create a new dataframe that lists the number of binding sites for each TF, sorted from most sites to least sites. You will need to set two keyword arguments in `.sort_values()`: `by='site'` and `ascending=False`.

In [ ]:
# Answer here

**E4.5.** Fill out the function below so that the user can pass the name of any TF and get list of aligned sites back. Test that it works, e.g. on `tf='FNR'`, by getting a list of sites and making an information logo. Also test that it fails when it is supposed to.

In [ ]:
# Now let's turn this into a function 
def get_tf_sites(tf):
   
    # Load database
    df = pd.read_csv("data/binding_site_db.txt", sep='\t', comment='#',
                 header=None, usecols=[1,11], names=['tf','site']).dropna()
    
    # 
    # Fill in stuff here
    # 
    
    # Get sequence alignment and return it
    return tf_df['site']
